# Sistemas Operacionais
## Laboratório 02 - Threads e comunicação entre processos

### **Introdução**

Neste laboratório aprenderemos a realizar a comunicação entre processos usando o padrão POSIX e a linguagem C no Linux. O foco desta aula será no aspecto prático do conteúdo, todavia um breve resumo da teoria será exposto, quando necessário, para auxiliar na execução dos exercícios.

Ao final desta aula, o estudante deverá ser capaz de:
* Fazer o estabelecimento de uma área de memória compartilhada;
* Fazer a manipulação de variáveis usando múltiplos processos.

### **Instruções**

Este laboratório foi elaborado utilizando a ferramenta Google Colab, com o objetivo de evitar a necessidade de instalação de programas adicionais ou máquinas virtuais. Para completá-lo, espera-se que o aluno siga os comandos de cada etapa, buscando compreender o impacto das execuções no sistema. Recomenda-se também que o aluno introduza pequenas modificações nos códigos fornecidos, como forma de testar e consolidar seu aprendizado.

O material inclui 3 exercícios práticos e corresponde a 2,5 pontos na nota da unidade. **A entrega deverá ser realizada via SIGAA, sendo necessário enviar apenas os códigos que respondem aos exercícios. Recomenda-se que estes sejam organizados em um arquivo com extensão *.ipynb*. O envio dos códigos em formato PDF ou em imagem, por dificultar a correção, acarretará um desconto de 1 ponto na nota final do laboratório.**

A professora não fornecerá orientações sobre programação até o prazo final do exercício; contudo, o aluno poderá consultar todos os materiais que julgar necessários.


### **Parte 1 - Aprendendo a implementar a comunicação entre processos**
Como vimos em nossas aulas teóricas, os processos têm espaços de endereçamentos próprios, os quais, por via de regra, não se comunicam entre si. Quando um processo deseja trocar informações com outros processos, ele pode usar dois modelos de comunicação: transmissão de mensagens ou memória compartilhada.

No modelo de comunicação via memória compartilhada, os processos comunicam ao sistema operacional que desejam estabelecer uma região de memória que poderá ser acessada por mais de um processo. Isto é feito por meio das chamadas de sistema *shmget*, que cria a memória compartilhada, e *shmat* que anexa a memória compartilhada ao espaço de endereçamento do processo, retornando um ponteiro de acesso à tal memória.

Ao final da comunicação, os processos podem desanexar a memória por meio da chamada de sistema *shmdt* e removê-la via *shmctl*. Observe a utilização de tais chamadas de sistema no código abaixo, no qual o processo pai lê o que o filho escreveu em um espaço de memória compartilhada.

In [ ]:
%%writefile sharedMemory.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
  // Criando o segmento de memoria compartilhada com o tamanho de 100 chars
  int segment_id=shmget(IPC_PRIVATE, 100*sizeof(char),S_IRUSR|S_IWUSR);

  // Criando o ponteiro que vai apontar para o endereco de memoria compartilhada
  char *sharedMemory;

  // Funcao usada para criar um novo processo
  pid_t pid=fork();

  if(pid<0){ // Erro na criacao do processo
    printf("Sou o processo %d e nao consegui criar um novo processo.\n",getpid());
    return 1;
  }else if(pid==0){ // Processo filho
    // Anexando o segmento de memoria compartilhada
    sharedMemory=(char *) shmat(segment_id, NULL,0);

    // Escrevendo na memoria compartilhada
    sprintf(sharedMemory, "Ola, processo pai, sou o seu filho e o meu ID e %d.",getpid());

    // Desanexando a memoria compartilhada
    shmdt(sharedMemory);

    // Encerrando o processo filho
    exit(0);
  }else{ // Processo pai
    // Fazendo o pai esperar que o filho termine de escrever
    wait(NULL);

    // Salvando o ponteiro que aponta para a memoria compartilhada
    sharedMemory=(char *) shmat(segment_id, NULL,0);

    // Imprimindo mensagem deixada pelo processo filho
    printf("%s\n",sharedMemory);

    // Removendo o segmento de memoria compartilhada
    shmctl(segment_id, IPC_RMID,NULL);
  }
  return 0;
}


Overwriting sharedMemory.c


In [ ]:
!gcc ./sharedMemory.c -o sharedMemory
!./sharedMemory

Ola, processo pai, sou o seu filho e o meu ID e 1286.


### **Parte 2 - Exercícios**
**Exercício 1** (0,75)

Modifique o código acima para fazer com que dois processos filhos alterem uma memória compartilhada contendo um inteiro. Faça o processo pai esperar por seus dois filhos para depois imprimir o valor da variável na tela. Dica: olhe o código do exercício 3 do Laboratório 1.

---


**Exercício 2** (0,75)

Utilizando comunicação entre processos, crie um código que escreva a Sequência de Fibonacci, com o parâmetro n sendo uma variável global. Tal sequência deve ser gerada por um processo filho, mas deve ser impressa na tela pelo processo pai. Dica: olhe o código do desafio do Laboratório 1.


---



**Exercício 3** (1,00)

Usando comunicação entre processos, escreva um código que realiza a soma de *n* números naturais, sendo *n* uma variável global. Cada processo deve fazer apenas a soma de dois números e o resultado final deve ser retornado pelo primeiro processo. Por exemplo, o processo 1 deve fazer a soma de 0+1, o processo 2 vai pegar esse resultado e somar com 2, e assim por diante.


---



# Exercícios

## Exercício 1

In [ ]:
%%writefile exercicio1.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
    // Criando o segmento de memoria compartilhada com o tamanho de 100 chars
    int segment_id = shmget(IPC_PRIVATE, 10*sizeof(int), S_IRUSR|S_IWUSR);

    // Criando o ponteiro que vai apontar para o endereco de memoria compartilhada
    int* sharedMemory = (int*) shmat(segment_id, NULL, 0);
    *sharedMemory = 0;

    // Funcao usada para criar um novo processo
    pid_t pid1 = fork();

    if (pid1 < 0) { // Erro na criacao do processo
        printf("Sou o processo %d e nao consegui criar um novo processo.\n", getpid());
        return 1;
    } else if (pid1 == 0) { // Processo filho
        // Escrevendo na memoria compartilhada
        *sharedMemory += 1;

        // Desanexando a memoria compartilhada
        shmdt(sharedMemory);

        // Encerrando o primeiro processo filho
        exit(0);
    } else { // Processo pai
        // Fazendo o pai esperar que o primeiro filho termine de alterar o numero
        wait(NULL);

        // Cria um novo filho
        pid_t pid2 = fork();

        if (pid2 < 0) {
            printf("Sou o processo %d e nao consegui criar um novo processo.\n",getpid());
            return 1;
        } else if (pid2 == 0 ) {
            // Escrevendo na memoria compartilhada
            *sharedMemory += 1;

            // Desanexando a memoria compartilhada
            shmdt(sharedMemory);

            // Encerrando o segundo processo filho
            exit(0);
        } else {
            // Espera o segundo filho terminar tambem
            wait(NULL);

            // Imprimindo o numero manipulado pelos processos filhos
            printf("%d\n", *sharedMemory);

            // Removendo o segmento de memoria compartilhada
            shmctl(segment_id, IPC_RMID, NULL);
        }
    }

    return 0;
}


Overwriting exercicio1.c


In [ ]:
!gcc ./exercicio1.c -o exercicio1
!./exercicio1

2


## Exercício 2

In [ ]:
%%writefile exercicio2.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int fibonacci(int n) {
    int antePen, pen = 0, atual = 1;

    if (n == 0 || n == 1) {
        return n;
    }

    for (int i = 2; i <= n; i++) {
        antePen = pen;
        pen = atual;
        atual = antePen + pen;
    }

    return atual;
}

int main(int argc, char **argv)
{
    int tamanhoFib;

    printf("Digite o tamanho da sequencia de fibonacci desejada: ");
    scanf("%d", &tamanhoFib);

    if (tamanhoFib < 0) {
        printf("Digite um numero nao negativo");
        return 1;
    }

    // Criando o segmento de memoria compartilhada
    int segment_id = shmget(IPC_PRIVATE, tamanhoFib * sizeof(int), S_IRUSR | S_IWUSR);

    // Funcao usada para criar um novo processo
    pid_t pid = fork();

    if (pid < 0) { // Erro na criacao do processo
        printf("Sou o processo %d e nao consegui criar um novo processo.\n", getpid());
        return 1;
    } else if (pid == 0) { // Processo filho
        // Anexando o segmento de memoria compartilhada
        int* sharedMemory = (int*) shmat(segment_id, NULL, 0);

        for (int i = 0; i < tamanhoFib; i++) {
            sharedMemory[i] = fibonacci(i);
        }

        // Desanexando a memoria compartilhada
        shmdt(sharedMemory);

        // Encerrando o processo filho
        exit(0);
    } else { // Processo pai
        // Fazendo o pai esperar que o primeiro filho termine de alterar o numero
        wait(NULL);

        // Anexando o segmento de memoria compartilhada
        int* sharedMemory = (int*) shmat(segment_id, NULL, 0);

        // Imprimindo a sequencia gerada pelo processo filho
        printf("Sequencia de fibonacci: \n");
        for (int i = 0; i < tamanhoFib; i++) {
            printf("%d ", sharedMemory[i]);
        }

        // Removendo o segmento de memoria compartilhada
        shmctl(segment_id, IPC_RMID, NULL);
    }

    return 0;
}


Overwriting exercicio2.c


In [ ]:
!gcc ./exercicio2.c -o exercicio2
!./exercicio2

Digite o tamanho da sequencia de fibonacci desejada: 20
Sequencia de fibonacci: 
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 

## Exercício 3

In [ ]:
%%writefile exercicio2.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
    int n;

    printf("Digite a quantidade de numeros a serem somados: ");
    scanf("%d", &n);

    if (n < 0) {
        printf("Digite um numero nao negativo");
        return 1;
    }

    // Criando o segmento de memoria compartilhada
    int segment_id = shmget(IPC_PRIVATE, 100 * sizeof(int), S_IRUSR | S_IWUSR);

    int* sharedMemory = (int*) shmat(segment_id, NULL, 0);
    *sharedMemory = 0;

    for (int i = 1; i <= n; i++) {
        pid_t pid = fork();

        if (pid < 0) {
            printf("Erro ao gerar o %d° processo", n);
            return 1;
        } else if (pid == 0) {
            *sharedMemory += i;

            // Todos os processos filhos morrerao aqui
            shmdt(sharedMemory);
            exit(0);
        }
    }

    for (int i = 0; i < n; i++) {
        wait(NULL);
    }

    printf("O resultado total da soma e: %d", *sharedMemory);

    shmctl(segment_id, IPC_RMID, NULL);

    return 0;
}


Overwriting exercicio2.c


In [ ]:
!gcc ./exercicio2.c -o exercicio2
!./exercicio2

Digite a quantidade de numeros a serem somados: 6
O resultado total da soma e: 21